# Mask sentences

**module:** lexsub.run_generate_masked_sentences
```
python -m lexsub.run_generate_masked_sentences --configs $JSON_CONFIGS_FILE_PATH --workers 12
```


### create json configs

In [ ]:
import json
import os
from glob import glob

In [ ]:
def mask_template(name, exps_dir, exp, output_dir, substitute_lu, substitute_role, noun_max, ibn):
    return {"name":name,
            "args":{
                "input_exp": os.path.join(exps_dir, exp),
                "output_exp": os.path.join(output_dir, name),
                "substitute_lu":substitute_lu,
                "substitute_role":substitute_role,
                "noun_max":f'{noun_max}', 
                "ibn":ibn
                }
             }

In [ ]:
import json
# -----------------------------------------------------
seed_exps = [
    '01ExPerSent_verbs_rand01'
]
exps_dir='nExPerSent_verbs_randAllExps'
output_dir = 'expanded_nExPerSent_verbs_randAllExps'
json_file = 'masking_nExPerSent_verbs_randAllExps'
# # ---------------------------------------------------
# exps_dir = "nPc_verbs_randAllExps"
# seed_exps = [
#     '010pc_verbs',
#     '020pc_verbs',
#     '030pc_verbs',
#     '040pc_verbs',
#     '050pc_verbs',
#     '100pc_verbs'
# ]
# output_dir = 'expanded_nPc_verbs_randAllExps'
# json_file = 'masking_nPc_verbs_randAllExps'
# ---------------------------------------------------
# seed_exps = [
#     '01ExPerSent_nouns_rand01',
# ]
# exps_dir='nExPerSent_nouns_randAllExps'
# output_dir = 'expanded_nExPerSent_nouns_randAllExps'
# json_file = 'masking_nExPerSent_nouns_randAllExps'

# # ---------------------------------------------------
# seed_exps = [
# '010pc_nouns',
# '020pc_nouns',
# '030pc_nouns',
# '040pc_nouns',
# '050pc_nouns',
# '100pc_nouns'
# ]
# exps_dir = "nPc_nouns_randAllExps"
# output_dir = 'expanded_nPc_nouns_randAllExps'
# json_file = 'masking_nPc_nouns_randAllExps'
# # ---------------------------------------------------


configs_output = 'configurations/configs_augment'
nouns = [0.1,0.3,0.5]

# nouns = [0.3, 0.5]


expanded_configs = [
    'lu', 
    'roles', 
    'nouns',
    'lu-roles-nouns',
]
# --------------------------------------------------------------------    
exp_configs= []

for exp in seed_exps:
    
    base_name = f'{exp}_expanded'
    
    for config in expanded_configs:
        
        name = base_name
        
#         print(config)
        substitute_lu = False
        substitute_role = False
        ibn = True
        noun_max = 0
        
        if 'lu' in config:
            
            substitute_lu = True
            name = f'{base_name}_lu'
        
        if 'roles' in config:
            substitute_role = True
            name = f'{name}_roles'
        
        if 'nouns' in config:     
              
            for noun_max in nouns:
                if noun_max<=0: continue 
                
                name_n = f'{name}_nouns-{int(100*noun_max)}pc'
                ibn = True
                if 'nouns-ooa' in config:
                    name_n = f'{name_n}-ooa'
                    ibn = False
                    
                exp_configs.append(mask_template(name_n, exps_dir, exp, output_dir, 
                                            substitute_lu, substitute_role, noun_max, ibn))    

        elif name!=base_name:
            exp_configs.append(mask_template(name, exps_dir, exp, output_dir, 
                                            substitute_lu, substitute_role, noun_max, ibn))

                            
 
print(f'json file:{json_file}')
print(len(exp_configs))
exp_names =[exp['name'] for exp in exp_configs]
# print(','.join(exp_names))

with open(f'{configs_output}/{json_file}.json', 'w') as fp:
        json.dump(exp_configs, fp, indent=4)
        
    
exp_configs[0]    

# Postprocess and do final augmentation

**module:** lexsub.run_augment_conll_sentences
```
python -m lexsub.run_augment_conll_sentences --configs $JSON_CONFIGS_PATH --workers 12
```

### create json configs

In [ ]:
def augmentation_template(name, exps_dir, exp, output_dir, model_type, proc_funcs, parser, match_gold, pipeline, N, postprocess, final_preds_path):
#     print(name)
    return {"name":name,
            "args":{
                "input_exp": os.path.join(exps_dir, exp),
                "output_exp": os.path.join(output_dir, name),
                "preds_model": model_type,                                
                "proc_funcs":proc_funcs,  
                "parser":parser,
                "match_lugold":match_gold[0],                                
                "match_rolegold":match_gold[1],
                "pipeline":pipeline,
                "N":N,
                "postprocess":postprocess,
                "final_preds_path":final_preds_path
                }
             }


In [ ]:
import os
import json

# ---------------------------------------------------
seed_exps = [
    '01ExPerSent_verbs_rand01',
]
exps_dir='nExPerSent_verbs_randAllExps'
output_dir = 'expanded_nExPerSent_verbs_randAllExps'

# # ---------------------------------------------------

# seed_exps = [
#     '010pc_verbs',
#     '020pc_verbs',
#     '030pc_verbs',
#     '040pc_verbs',
#     '050pc_verbs',
#     '100pc_verbs'
# ]
# exps_dir = "nPc_verbs_randAllExps"
# output_dir = 'expanded_nPc_verbs_randAllExps'
# # ---------------------------------------------------
# seed_exps = [
#     '01ExPerSent_nouns_rand01'
# ]
# exps_dir='nExPerSent_nouns_randAllExps'
# output_dir = 'expanded_nExPerSent_nouns_randAllExps'

# # ---------------------------------------------------

# seed_exps = [
# '010pc_nouns',
# '020pc_nouns',
# '030pc_nouns',
# '040pc_nouns',
# '050pc_nouns',
# '100pc_nouns'
# ]
# exps_dir = "nPc_nouns_randAllExps"
# output_dir = 'expanded_nPc_nouns_randAllExps'
# ---------------------------------------------
# nouns = [0.3,0.5]
nouns = [0.1,0.3,0.5]


preds_models = [
# 'dt,
# 'embs',
# 'melamud_baladd',
'bert',
'xlnet_embs_hypers'
]
model = ''

match_gold, gold_prefix = [True, True], 'lugold_rolegold'
if gold_prefix != '': gold_prefix = gold_prefix+'_'
N = 2
postprocess = False
parser = 'nltk'
# proc_funcs = 'nolemma'
proc_funcs = 'nolemma_role_stopwords'

pipeline = f'{gold_prefix}{parser}_{proc_funcs}_N{N}'
preds_pipeline = f'{parser}_{proc_funcs}' 

# -----------------------------------------------------------------
json_file = f'{model}{output_dir}_{pipeline}.json'
configs_output = "configurations/configs_augment" 
# --------------------------------------------------------------------    
expanded_configs = [
#     'lu', 
    'roles', 
#     'nouns',
    'lu-roles-nouns',
]

exp_configs= []

for exp in seed_exps:
#     print(exp)
    for model_type in preds_models:
        
        base_name = f'{exp}_expanded'
        
        for config in expanded_configs:
            
#             print(config)
            
            if not 'nouns' in config: 
                if config == 'lu':
                    name = f'{base_name}_lu'
                elif config == 'roles':
                    name = f'{base_name}_roles'
                elif config == 'lu-roles':
                    name = f'{base_name}_lu_roles'
                else: continue
#                 print(name)
                final_preds_path = f'{output_dir}/{name}/{model_type}/{preds_pipeline}_final_predictions.pkl'
                exp_configs.append(augmentation_template(name, exps_dir, exp, output_dir, model_type, 
                                              proc_funcs, parser,match_gold, pipeline, N, postprocess, final_preds_path))
            else: 
                
                for noun_max in nouns:
                    if noun_max<=0: continue        

                    if config == 'nouns':
                        name = f'{base_name}_nouns-{int(100*noun_max)}pc' 
                    elif config == 'nouns-ooa':
                        name = f'{base_name}_nouns-{int(100*noun_max)}pc-ooa'
                    elif config == 'roles-nouns':
                        name = f'{base_name}_roles_nouns-{int(100*noun_max)}pc'
                    elif config == 'roles-nouns-ooa':
                        name = f'{base_name}_roles_nouns-{int(100*noun_max)}pc-ooa'
                    elif config == 'lu-nouns':
                        name = f'{base_name}_lu-nouns-{int(100*noun_max)}pc'
                    elif config == 'lu-nouns-ooa':
                        name = f'{base_name}_lu_nouns-{int(100*noun_max)}pc-ooa'
                    elif config == 'lu-roles-nouns':
                        name = f'{base_name}_lu_roles_nouns-{int(100*noun_max)}pc'
                    elif config == 'lu-roles-nouns-ooa':
                        name = f'{base_name}_lu_roles_nouns-{int(100*noun_max)}pc-ooa'
                    else:
                        continue
                    
#                     print(name)
                    final_preds_path = f'{output_dir}/{name}/{model_type}/{preds_pipeline}_final_predictions.pkl'
                    exp_configs.append(augmentation_template(name, exps_dir, exp, output_dir, model_type, 
                                                proc_funcs, parser,match_gold, pipeline, N, postprocess, final_preds_path))

        
print(f'json file: {json_file}')
print(len(exp_configs))
exp_names =[exp['name'] for exp in exp_configs]
# print(','.join(exp_names))

with open(f'{configs_output}/{json_file}', 'w') as fp:
        json.dump(exp_configs, fp, indent=4)
        
        
exp_configs[0]        